In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import streamlit as st
from energyConsumption import energyConsumption

**Author: K.Bodrova:**

In [37]:
# Function to read and clean data from CSV files
def read_and_clean_data(file_production, file_consumption):
    # Read data from CSV files
    production_df = pd.read_csv(file_production, delimiter=';')
    consumption_df = pd.read_csv(file_consumption, delimiter=';')

    # Columns
    DATE = 'Datum'
    STARTTIME = 'Anfang'
    BIOMAS = 'Biomasse [MWh] Originalauflösungen'
    HYDROELECTRIC = 'Wasserkraft [MWh] Originalauflösungen'
    WIND_OFFSHORE = 'Wind Offshore [MWh] Originalauflösungen'
    WIND_ONSHORE = 'Wind Onshore [MWh] Originalauflösungen'
    PHOTOVOLTAIC = 'Photovoltaik [MWh] Originalauflösungen'
    OTHER_RENEWABLE = 'Sonstige Erneuerbare [MWh] Originalauflösungen'
    CONSUMPTION = 'Gesamt (Netzlast) [MWh] Originalauflösungen'

    # Convert date columns to DateTime objects
    production_df['Date'] = pd.to_datetime(production_df[DATE], format='%d.%m.%Y')
    production_df['Starttime'] = pd.to_datetime(production_df[STARTTIME], format='%H:%M')
    consumption_df['Date'] = pd.to_datetime(consumption_df[DATE], format='%d.%m.%Y')
    consumption_df['Starttime'] = pd.to_datetime(consumption_df[STARTTIME], format='%H:%M')
    
    # Clean data formats for renewable energies
    production_df['Biomass'] = production_df[BIOMAS].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)
    production_df['Hydroelectric'] = production_df[HYDROELECTRIC].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)
    production_df['Wind Offshore'] = production_df[WIND_OFFSHORE].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)
    production_df['Wind Onshore'] = production_df[WIND_ONSHORE].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)
    production_df['Photovoltaic'] = production_df[PHOTOVOLTAIC].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)
    production_df['Other Renewable'] = production_df[OTHER_RENEWABLE].str.replace(".", "").str.replace(",", ".").replace('-', 0).astype(float)

    # Clean data formats for total consumption
    consumption_df['Consumption'] = consumption_df[CONSUMPTION].str.replace(".", "").str.replace(",", ".").astype(float)

    # Create a new column for total production
    production_df['Total Production'] = production_df[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum(axis=1)

    # Gruppierung der Produktionsdaten nach Jahr und Summierung der erneuerbaren Energietypen
    production_by_type_and_year = production_df.groupby(production_df['Date'].dt.year)[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum()
 
   
    pd.options.display.float_format = '{:.2f}'.format  # Set Pandas to display floating-point numbers with two decimal places

    data_by_year = {}  # Aggregation der Daten nach Jahren und Speicherung in einem Dictionary

    for year, data in production_df.groupby(production_df['Date'].dt.year):
        production_data = data[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum()
        consumption_data = consumption_df[consumption_df['Date'].dt.year == year]['Consumption']
        total_consumption = consumption_data.sum()
        data_by_year[year] = {
            'Production': production_data.sum(),
            'Consumption': total_consumption,
            'Biomass': production_data['Biomass'],
            'Hydroelectric': production_data['Hydroelectric'],
            'Wind Offshore': production_data['Wind Offshore'],
            'Wind Onshore': production_data['Wind Onshore'],
            'Photovoltaic': production_data['Photovoltaic'],
            'Other Renewable': production_data['Other Renewable']
        }

    total_renewable_production = production_df[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum(axis=1)
    total_consumption = consumption_df['Consumption']

    return production_df, consumption_df, total_renewable_production, total_consumption, data_by_year

**Authors: M.Lauterbach, K.Bodrova:**

In [38]:
installed_power_dict = {2020: 122603, 2021: 129551, 2022: 133808}

def find_dark_lulls_and_for_years(selected_date, production_df, dark_lulls_dict, columns_to_clean):
    
    # Get the year of the selected date
    year = selected_date.year
    
    # Installed power for the corresponding year
    installed_power = installed_power_dict.get(year, None)
    
    if installed_power is None:
        print(f"No installed power found for the year {year}.")
        return None
    
    # Filter data for the selected date
    selected_production = production_df[production_df['Date'] == selected_date]
    
    # Sum the renewable energy production for the selected date
    total_renewable_production_selected_date = selected_production[columns_to_clean].sum(axis=1).sum()
    
    # Compare with installed power for different thresholds
    threshold_10_percent = installed_power * 0.1
    threshold_20_percent = installed_power * 0.2
    
    if total_renewable_production_selected_date/24 < threshold_10_percent:
        dark_lulls_dict["up to 10%"].append(selected_date)
    elif total_renewable_production_selected_date/24 < threshold_20_percent:
        dark_lulls_dict["up to 20%"].append(selected_date)
    else:
        return None

def find_dark_lulls_for_years(production_df, columns_to_clean):
    # Loop through all days in the years 2020 to 2022
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2022, 12, 31)

    dark_lulls_dict = {"up to 10%": [], "up to 20%": []}
    current_date = start_date
    
    while current_date <= end_date:
        find_dark_lulls_and_for_years(current_date, production_df, dark_lulls_dict, columns_to_clean)
        current_date += pd.DateOffset(days=1)
    
    # Sort lists by date
    for label, days_list in dark_lulls_dict.items():
        dark_lulls_dict[label] = sorted(days_list)
    

    # Display the sorted lists
    print("\nList of days up to 10%:")
    for day in dark_lulls_dict["up to 10%"]:
        print(day.strftime('%d.%m.%Y'))

    print("\nList of days up to 20%:")
    for day in dark_lulls_dict["up to 20%"]:
        print(day.strftime('%d.%m.%Y'))

    print("\nNumber of days up to 10%:", len(dark_lulls_dict["up to 10%"]))
    print("Number of days up to 20%:", len(dark_lulls_dict["up to 20%"]))



**Author: K.Bodrova:**

In [39]:
# Funktion zur Berechnung und Anzeige des Histogramms für erneuerbare Anteile
def calculate_and_display_renewable_shares_histogram(total_renewable_production, total_consumption):
    # Berechnung der prozentualen Anteile der erneuerbaren Energieerzeugung am Gesamtverbrauch
    percent_renewable = total_renewable_production / total_consumption * 100 

    counts, intervals = np.histogram(percent_renewable, bins = np.arange(0, 111, 1))  # Use NumPy to calculate the histogram of the percentage distribution

    x = intervals[:-1]                               # Define the x-axis values as the bin edges
    labels = [f'{i}%' for i in range(0, 111, 1)]     # Create labels for x-axis ticks (von 0 bis 111 in Einzelnschritten)

    fig = go.Figure(data=[go.Bar(x=x, y=counts)])    # Create a bar chart using Plotly

    fig.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(0, 111, 5)), ticktext=labels[::5]))  # X-axis label settings

    # Title and axis labels settings
    fig.update_layout(title='Anzahl der Viertelstunden in Jahren 2020-2022 mit 0-110 % EE-Anteilen',
                  xaxis_title='Prozentsatz erneuerbarer Energie',
                  yaxis_title='Anzahl der Viertelstunden')

    fig.show()
 

**Author: K. Bodrova, Diagram: O. Pearse-Danker:**

In [40]:
# Function to plot energy consumption and production for a selected date
def plot_energy_consumption_and_production(production_df, consumption_df, columns_to_clean):
    # Prompt user to enter the selected date
    selected_date_str = input("Enter the selected date (format: YYYY-MM-DD): ")
    
    try:
        selected_date = pd.to_datetime(selected_date_str)
    except ValueError:
        print("Invalid date format. Please use the format YYYY-MM-DD.")
        return

    # Filter data for the selected date
    selected_production = production_df[production_df['Date'] == selected_date]
    selected_consumption = consumption_df[consumption_df['Date'] == selected_date]

    total_renewable_production_selected_date = selected_production[columns_to_clean].sum(axis=1).sum()
    print(f"Summe der erneuerbaren Energien am {selected_date}: {total_renewable_production_selected_date} MWh")

    # Plotting with Plotly
    # Create a new Plotly subplot figure
    fig = make_subplots()

    # Add the energy consumption trace
    fig.add_trace(
        go.Scatter(
            x=selected_consumption['Starttime'].dt.strftime('%H:%M'),
            y=selected_consumption['Consumption'],
            mode='lines',
            name='Total Consumption',
            fill='tozeroy'
        )
    )

    # Add the renewable energy production trace
    fig.add_trace(
        go.Scatter(
            x=selected_production['Starttime'].dt.strftime('%H:%M'),
            y=selected_production['Total Production'],
            mode='lines',
            name='Total Renewable Production',
            fill='tozeroy'
        )
    )

    fig.update_layout(
        title=f'Energy Production and Consumption on {selected_date}',
        xaxis=dict(title='Time (hours)'),
        yaxis=dict(title='Energy (MWh)'),
        showlegend=True
    )

    # Show the plot
    fig.show()

**Author: K.Bodrova:**

In [41]:
# Funktion zur Berechnung und Anzeige der aggregierten Daten pro Jahr
def calculate_and_display_data_by_year(data_by_year):
    for year, data in data_by_year.items():
        print(f"Jahr: {year}")
        print(f"Gesamte erneuerbare Energieproduktion: {data['Production']} MWh")
        print(f"Gesamtverbrauch: {data['Consumption']} MWh")
        print(f"Biomasse: {data['Biomass']} MWh")
        print(f"Wasserkraft: {data['Hydroelectric']} MWh")
        print(f"Wind Offshore: {data['Wind Offshore']} MWh")
        print(f"Wind Onshore: {data['Wind Onshore']} MWh")
        print(f"Photovoltaik: {data['Photovoltaic']} MWh")
        print(f"Andere erneuerbare Energien: {data['Other Renewable']} MWh")
        print()


**Authors: L.Dorda, N.Clasen, B.Wolf:**

In [42]:
# Function to process and plot data for the year 2030
def process_and_plot_2030_data(production_df, consumption_df):
    # Abfrage datum
    while True:
        selected_date_str = input("Bitte geben Sie ein Datum IM JAHR 2030 im Format TT.MM.JJJJ ein: ")
        selected_date = datetime.strptime(selected_date_str, "%d.%m.%Y")

        if selected_date.year == 2030:
            break
        else:
            print("Bitte geben Sie ein Datum aus dem Jahr 2030 an.")

    # Define a dataframe of the production of 2022
    production_2022df = production_df[production_df['Date'].dt.year == 2022]
    prognoseErzeugung2030df = production_2022df.copy()
    prognoseErzeugung2030df['Date'] = prognoseErzeugung2030df['Date'].map(lambda x: x.replace(year=2030))

    # Define a dataframe of the consumption of 2022
    consumption_2022df = consumption_df[consumption_df['Date'].dt.year == 2022]
    prognoseVerbrauch2030df = consumption_2022df.copy()
    prognoseVerbrauch2030df['Date'] = prognoseVerbrauch2030df['Date'].map(lambda x: x.replace(year=2030))

    # CHANGE LATER Define the factors Verbrauch 2022 to 2030 von Bjarne Noah
    # Verbrauch2022_2030_factor = 1.157  # assuming consumption will increase by 10% from 2022 compared to 2030

    def scale_2030_factorsConsumption(df, Verbrauch2022_2030_factor):
        df_copy = df.copy()
        df_copy['Consumption'] *= Verbrauch2022_2030_factor
        return df_copy

    '''
    #LATER put in Verbrauch 2030 fron noah&bjarne
    prognoseVerbrauch2030df = energyConsumption(consumption_df)
    '''

    # Define the factors
    # müssen noch angepasst werden
    windonshore_2030_factor = 2.03563  # assuming Wind Onshore will increase by 203% from 2022 compared to 2030
    windoffshore_2030_factor = 3.76979  # assuming Wind Offshore will 376% increase
    pv_2030_factor = 3.5593  # assuming PV will increase by 350%

    def scale_2030_factors(df, windonshore_factor, windoffshore_factor, pv_factor):
        df_copy = df.copy()
        df_copy['Wind Onshore'] *= windonshore_factor
        df_copy['Wind Offshore'] *= windoffshore_factor
        df_copy['Photovoltaic'] *= pv_factor
        df_copy['Total Production'] = df_copy[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum(axis=1)
        return df_copy

    # Scale the data by the factors
    scaled_production_df = scale_2030_factors(prognoseErzeugung2030df, windonshore_2030_factor, windoffshore_2030_factor,
                                              pv_2030_factor)

    # Filter the data for the selected date
    scaled_selected_production_df = scaled_production_df[scaled_production_df['Date'] == selected_date]

    verbrauch2030df = energyConsumption(consumption_df)

    # scaled_consumption_df = scale_2030_factorsConsumption(prognoseVerbrauch2030df, Verbrauch2022_2030_factor)

    # Filter the data for the selected date
    # scaled_selected_consumption_df = scaled_consumption_df[scaled_consumption_df[DATE] == selected_date]

    ### subplot only consumption 2030

    ## Plotly daily production and consumption 2030

    selected_consumption2030df = verbrauch2030df[verbrauch2030df['Date'] == selected_date]
    scaled_selected_production_df = scaled_selected_production_df[scaled_selected_production_df['Date'] == selected_date]

    fig = make_subplots()

    # Add the energy consumption trace
    fig.add_trace(
        go.Scatter(
            x=selected_consumption2030df['Starttime'].dt.strftime('%H:%M'),
            y=selected_consumption2030df['Verbrauch [MWh]'],
            mode='lines',
            name='Total Consumption',
            fill='tozeroy'
        )
    )

    # Add the renewable energy production trace
    fig.add_trace(
        go.Scatter(
            x=scaled_selected_production_df['Starttime'].dt.strftime('%H:%M'),
            y=scaled_selected_production_df['Total Production'],
            mode='lines',
            name='Total Renewable Production',
            fill='tozeroy'
        )
    )

    fig.update_layout(
        title=f'Energy Production and Consumption on {selected_date}',
        xaxis=dict(title='Time (hours)'),
        yaxis=dict(title='Energy (MWh)'),
        showlegend=True
    )

    fig.show()


    # code to do 2030 quarter hours
    total_scaled_renewable_production = scaled_production_df[['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable']].sum(axis=1)
    total_consumption = verbrauch2030df['Verbrauch [MWh]']

    # Berechnung der prozentualen Anteile der erneuerbaren Energieerzeugung am Gesamtverbrauch
    percent_renewable = total_scaled_renewable_production / total_consumption * 100

    counts, intervals = np.histogram(percent_renewable, bins=np.arange(0, 330, 1))  # Use NumPy to calculate the histogram of the percentage distribution

    x = intervals[:-1]  # Define the x-axis values as the bin edges
    labels = [f'{i}%' for i in range(0, 330, 1)]  # Create labels for x-axis ticks (von 0 bis 111 in Einzelnschritten)

    fig = go.Figure(data=[go.Bar(x=x, y=counts)])  # Create a bar chart using Plotly
    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=list(range(0, 330, 5)), ticktext=labels[::5]))  # X-axis label settings

    # Title and axis labels settings
    fig.update_layout(title='Anzahl der Viertelstunden im Jahren 2030 mit 0-330 % EE-Anteil',
                  xaxis_title='Prozentsatz erneuerbarer Energie',
                  yaxis_title='Anzahl der Viertelstunden')

    fig.show()

    # how many quarter hours are in scaled_production_df
    print("soviele VS sind in scaled_production_df:")
    print(len(scaled_selected_production_df))
    print("Viertelstunden aus 2030 expected 35040 hier: ")

    # Berechnen Sie den prozentualen Anteil der erneuerbaren Energieerzeugung am Verbrauch
    prozentualerAnteil = total_scaled_renewable_production / verbrauch2030df['Verbrauch [MWh]'] * 100
    print(percent_renewable)

    # Initialisieren Sie eine Liste für die Datensätze
    data = []

    # Iterieren Sie über die Prozentsätze von 0 bis 100
    for i in range(301):
        # Zählen Sie die Viertelstunden über oder gleich dem Prozentsatz
        anzahlViertelstundenProzent = len(percent_renewable[percent_renewable >= i])

    # Fügen Sie einen Datensatz zum Speichern in die Liste hinzu
    data.append({'Prozentsatz': i, 'Anzahl_Viertelstunden': anzahlViertelstundenProzent})

    # Erstellen Sie ein DataFrame aus der Liste
    result_df = pd.DataFrame(data)

    # Drucken Sie das erstellte DataFrame
    print(result_df)

    fig = go.Figure()

    # Fügen Sie einen Balken für die Anzahl der Viertelstunden für jeden Prozentsatz hinzu
    fig.add_trace(go.Bar(x=result_df['Prozentsatz'], y=result_df['Anzahl_Viertelstunden']))

    # Aktualisieren Sie das Layout für Titel und Achsenbeschriftungen
    fig.update_layout(
        title='Anzahl der Viertelstunden mit erneuerbarer Energieerzeugung über oder gleich dem Verbrauch',
        xaxis=dict(title='Prozentsatz erneuerbarer Energie'),
        yaxis=dict(title='Anzahl der Viertelstunden')
    )

    # Zeigen Sie den Plot an
    fig.show()





# Funktion zur Berechnung und Anzeige der aggregierten Daten pro Jahr
# Author: Bjarne, Noah
# Funktion
def energyConsumption(consumption_df):
    wärmepumpeHochrechnung2030 = wärmepumpe()
    eMobilitätHochrechnung2030 = eMobilität()

    print('\n', 'wärmepumpeHochrechnung2030', f"{wärmepumpeHochrechnung2030:,.0f}".replace(",", "."))
    print('\n', 'eMobilitätHochrechnung2030', f"{eMobilitätHochrechnung2030:,.0f}".replace(",", "."))

    verbrauch2022df = consumption_df[consumption_df['Date'].dt.year == 2022]
    prognose2030df = verbrauch2022df.copy()
    faktor = faktorRechnung(verbrauch2022df, wärmepumpeHochrechnung2030, eMobilitätHochrechnung2030)
    print(faktor)
    # Change the year in 'Datum' column to 2030
    prognose2030df['Date'] = prognose2030df['Date'].map(lambda x: x.replace(year=2030))

    prognose2030df['Verbrauch [MWh]'] = prognose2030df['Consumption'] * faktor

    combined_df = pd.concat([verbrauch2022df[['Starttime', 'Consumption']], prognose2030df[['Verbrauch [MWh]']]], axis=1)
    #print(combined_df[['Gesamt (Netzlast) [MWh] Originalauflösungen', 'Verbrauch [MWh]']])

    return prognose2030df


def wärmepumpe():
    wärmepumpeAnzahl2030 = 500000 * (2030 - 2023)  # 500k pro Jahr bis 2023

    heizstunden = 2000
    nennleistung = 15  # 15kW
    luftWasserVerhältnis = 206 / 236
    erdwärmeVerhältnis = 30 / 236
    luftWasserJAZ = 3.1
    erdwärmeJAZ = 4.1

    # Berechnung der einzelnen Pumpe
    luftWasserVerbrauch = wärmepumpeVerbrauchImJahr(heizstunden, nennleistung, luftWasserJAZ)  # in kW/h
    erdwärmeVerbrauch = wärmepumpeVerbrauchImJahr(heizstunden, nennleistung, erdwärmeJAZ)  # in kW/h

    luftWasserVerhältnisAnzahl = verhältnisAnzahl(wärmepumpeAnzahl2030, luftWasserVerhältnis)
    erdwärmeVerhältnisAnzahl = verhältnisAnzahl(wärmepumpeAnzahl2030, erdwärmeVerhältnis)

    return luftWasserVerbrauch * luftWasserVerhältnisAnzahl + erdwärmeVerbrauch * erdwärmeVerhältnisAnzahl  # kWh


# berechnung des Verbrauchs einer Wärmepumpe im Jahr
def wärmepumpeVerbrauchImJahr(heizstunden, nennleistung, jaz):
    # (Heizstunden * Nennleistung) / JAZ = Stromverbrauch pro Jahr
    return (heizstunden * nennleistung) / jaz


def verhältnisAnzahl(wärmepumpeAnzahl2030, verhältnis):
    return wärmepumpeAnzahl2030 * verhältnis


def eMobilität():
    eMobilität2030 = 15000000  # 15mio bis 20230
    eMobilitätBisher = 1307901  # 1.3 mio
    verbrauchPro100km = 21  # 21kWh
    kilometerProJahr = 15000  # 15.000km

    eMobilitätVerbrauch = (verbrauchPro100km / 100) * kilometerProJahr  # kWh

    return (eMobilität2030 - eMobilitätBisher) * eMobilitätVerbrauch


def faktorRechnung(verbrauch2022df, wärmepumpeHochrechnung2030, eMobilitätHochrechnung2030):
    gesamtVerbrauch2022 = otherFactors() + verbrauch2022df[
        'Consumption'].sum() * 1000  # mal1000 weil MWh -> kWh
    #print('\n', 'gesamtVerbrauch2022', f"{gesamtVerbrauch2022:,.0f}".replace(",", "."))
    return (gesamtVerbrauch2022 + wärmepumpeHochrechnung2030 + eMobilitätHochrechnung2030) / gesamtVerbrauch2022


def prognoseRechnung(verbrauch2022df, faktor):
    verbrauch2030df = verbrauch2022df['Verbrauch [kWh]'] * faktor
    return verbrauch2030df


def otherFactors():
    # positive Faktoren
    railway = 5000  # kWh
    batterieProdAndServerRooms = 13000  # kwh
    powerNetLoss = 1000

    # negative Faktoren
    efficiency = 51000
    other = 6000

    return railway + batterieProdAndServerRooms + powerNetLoss - efficiency - other


In [43]:
# Main program
def main():
    # File names
    file_production = 'Realisierte_Erzeugung_202001010000_202212312359_Viertelstunde.csv'
    file_consumption = 'Realisierter_Stromverbrauch_202001010000_202212312359_Viertelstunde.csv'

    # Read and clean data
    production_df, consumption_df, total_renewable_production, total_consumption, data_by_year = read_and_clean_data(file_production, file_consumption)
    
    # Find dark lulls for the years 2020-2022
    #find_dark_lulls_for_years(production_df, columns_to_clean=['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable'])

    # Berechnung und Anzeige des Histogramms für erneuerbare Anteile
    #calculate_and_display_renewable_shares_histogram(total_renewable_production, total_consumption)
    #calculate_and_display_data_by_year(data_by_year)

    #plot_energy_consumption_and_production(production_df, consumption_df, columns_to_clean=['Biomass', 'Hydroelectric', 'Wind Offshore', 'Wind Onshore', 'Photovoltaic', 'Other Renewable'])

   # process_and_plot_2030_data(production_df, consumption_df)
    
if __name__ == "__main__":
    main()